# Week 2B: Accessing data from Reddit

## Overview

Reddit is an online bulletin board system to host user-generated content; text, image, video, and audio `posts`. It is categorized into `subreddits` which are communities or user-groups meant to serve as a bulletin board on a specific topic or for a specific group of people. The users can `comment` on posts, which can then be `upvoted` or `downvoted` by other users. Each subreddit is moderated by `moderators` who try to enforce community rules to the comments and discussion therein.

## APIs

There are two APIs that are widely used to scrape data from Reddit
- Reddit API (https://www.reddit.com/dev/api/) - This is the most detailed API with endpoints that can enable us to find almost anything on Reddit. There is a Python wrapper, `praw`,  that helps us access this API (https://praw.readthedocs.io/en/stable/index.html). 
- Pushshift API (https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/) - Pushshift is a big-data storage and analytics project that enables access to the Reddit data albeit with some delay related to specific content (e.g, editing of comments might not be included instantly). It allows us to process data before accessing it, e.g., counting comments by specific users. In a raw Reddit API, we will need to do it locally, while Pushshift API enables us to do this without the hassle of downloading data. There is a Python wrapper, `psaw`, that helps us access Reddit data easily using Pushshit API (https://pypi.org/project/psaw/). Pushshift can be less stable than the pure Reddit API because it is a third-party application.

In this exercise, we will use `praw`, because when Andrew tested `psaw` this weekend it wasn't working. 

In [1]:
%%capture
# We call the Python package manager pip to install the praw package
# Using ! in ipython notebook runs the command in the bash shell and not in the python; try running !ls in a new cell
!pip install --upgrade praw

### Querying Pushshift API using Python


There are several parameters that can be passed to this search query. We will work through some of those parameters in this notebook. From here on, we will make use of `psaw` API. 

In [2]:
import pandas as pd
import praw


To use `praw` you need a Reddit user account. The startup instructions are here: https://praw.readthedocs.io/en/stable/getting_started/quick_start.html

Once you have a user account, and have made a `script` type app. Add your client secret, username, and user agent to a local json file using the code below. JSON is a file format which is very similar to a dictionary in Python, making it ideal for saving this sort of information. It is also easy to read and write.

This code creates a file on your computer with your login info. Do NOT share this file. This way, when you share the code, your account is safe. Anyone else could only run the code if they have their own login.

In [3]:
import json

# Fill this out with your own info, then delete your info and comment it out again once you have the file on your local machine.

secret =   {'client_id':"", #this should be right under the words 'personal use script'
            'client_secret':"", # this should be next to the word 'secret'
            'password':"", # this is your reddit password
            'user_agent':"testscript by u/username", # this is a description of the project
            'username':"", # this is your reddit username
           } 
 
with open("secret.json", "w") as f:
    json.dump(secret, f)

In [4]:
# Read your secret dictionary in from the file

with open("secret.json", "r") as f:
    secret = json.load(f)

In [5]:
# Use your secret dictionary to fill out the login info

reddit = praw.Reddit(
    client_id = secret['client_id'],
    client_secret = secret['client_secret'],
    username = secret['username'],
    password = secret['password'],
    user_agent = secret['user_agent']
)

If the login worked, the cell below should print out your username:

In [6]:
print(reddit.user.me())

None


As a precaution, we are setting read-only to True. Otherwise you can accidentally post to Reddit with your script.

In [7]:
reddit.read_only = True

<div class="alert alert-info">

**Exercise 0.1:** We are going to start with collecting the 100 hottest posts on Reddit from the subreddit Ask me Anything (IAmA; https://www.reddit.com/r/AMA/) and filtering for posts with more than 100 upvotes:
</div>

In [8]:
# To access a particular subreddit, use the .subreddit method of your reddit instance
# and pass in the name of the subreddit as the argument

subreddit = reddit.subreddit("AMA")

The `subreddit` object has several methods for getting posts; controversial, gilded, hot, new, rising, and top. For each of them, you can call the method and the number of responses you want.

The API returns a generator object, which is an iterator over the list. We can only access the elements of this list through iteration in a sequential manner, i.e, we can not index on this generator like `posts[0]`. You can convert generators to lists, but be careful of having enough memory available if you try to do so.

In [9]:
for post in subreddit.hot(limit=100):
    if (post.score > 100) and not (post.over_18):
        # Output: the post's title
        print(post.title)
        # Output: the post's score        
        print(post.score)
        # Output: the URL the submission points to or the submission's URL if it's a self post
        print(post.url)

ResponseException: received 401 HTTP response

<div class="alert alert-info">

**Exercise 0.2:** Make a pandas dataframe of the entries returned by the API
</div>

The code below is a list comprehension that loops through the generator and extracts relevant data for each matching Reddit post. It then turns that list into a Pandas DataFrame.

Note: Each element of `posts` is of type `praw.models.submission` which is a special object. You will want the `vars()` function to convert it to a dict.

In [ ]:
list_of_posts = [vars(p) for p in subreddit.hot(limit=100)]

df_posts = pd.DataFrame(list_of_posts)


<div class="alert alert-info">

**Exercise 0.3:** Now check for yourself the following - 

1. Number of rows and columns in the resulting dataframe 
2. The list of fieldnames that are returned by the API
</div>

In [ ]:
print("Shape (nb of rows, nb of columns):", df_posts.shape)

In [ ]:
# Which attributes do we now have access to?
df_posts.columns

### Exercise 1: Extracting texts from Reddit posts

<div class="alert alert-info">

**Exercise 1.1:** Let's now collect submissions in the `news` subreddit
- access the news subreddit and create an iterator of the most recent 1000 posts
- check if the post contains a keyword of your choice in either the title or body of the post
- make a dataframe where each row consists of a single entry returned by the API. 
- check the number of rows and columns in the resulting dataframe
</div>




In [ ]:
subreddit = reddit.subreddit("news")

In [ ]:
keyword = 'Ukraine'

submissions_list = []

for submission in subreddit.new(limit=1000):
    if (keyword in submission.title) or (keyword in submission.selftext):
        print(submission.title)
        submissions_list.append(vars(submission))
        
df_posts = pd.DataFrame(submissions_list)


<div class="alert alert-info">

**Exercise 1.2:** Let's look at some of the columns 
- have a look at the titles, number of comments, and the creation date
- Can you interpret the date column? Head over to https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html and use `pd.to_datetime` for conversion of this column to an appropriate human-readable format. 

</div>

In [104]:
df_posts[['title', 'num_comments', 'created_utc']]

,title,num_comments,created_utc
0,"Navy SEAL deserter killed in Ukraine, official...",238,1.674250e+09
1,Ukraine: US and European countries pledge heav...,14,1.674174e+09
2,Medvedev warns of nuclear war if Russia defeat...,959,1.674141e+09
3,Ukraine war: Ukraine admits retreat from front...,127,1.674134e+09
4,Berlin Won’t Allow Exports of German Tanks to ...,171,1.674089e+09
5,Ukraine crash: Ministers killed as helicopter ...,34,1.674032e+09
6,"Organizers ban Russian, Belarusian flags from ...",66,1.673947e+09
7,UK to supply tanks; Russian missiles hit acros...,31,1.673751e+09
8,Russia fires new waves of missiles at Ukraine ...,60,1.673734e+09
9,Rishi Sunak offers tanks to Ukraine to help ‘p...,50,1.673704e+09


`created_utc` is not interpretable as this is a common format that is shared across the systems in the world. Let's convert this to a time that humans can read. 

In [105]:
df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'], utc=True, unit='s')

In [106]:
df_posts[['title', 'num_comments', 'created_utc']].sample(10)

,title,num_comments,created_utc
2,Medvedev warns of nuclear war if Russia defeat...,959,2023-01-19 15:05:06+00:00
10,Ukraine war latest: UK will provide Challenger...,14,2023-01-14 12:26:34+00:00
7,UK to supply tanks; Russian missiles hit acros...,31,2023-01-15 02:46:00+00:00
8,Russia fires new waves of missiles at Ukraine ...,60,2023-01-14 22:03:25+00:00
1,Ukraine: US and European countries pledge heav...,14,2023-01-20 00:27:32+00:00
13,US to send Ukraine dozens of Bradleys in $2.85...,486,2023-01-05 19:31:53+00:00
9,Rishi Sunak offers tanks to Ukraine to help ‘p...,50,2023-01-14 13:54:43+00:00
4,Berlin Won’t Allow Exports of German Tanks to ...,171,2023-01-19 00:41:54+00:00
0,"Navy SEAL deserter killed in Ukraine, official...",238,2023-01-20 21:31:53+00:00
12,Ukraine denies Russian claim it killed 600 sol...,33,2023-01-09 04:53:17+00:00



<div class="alert alert-info">

**Exercise 1.3:** How would you list the usernames of the posters?

</div>


In [108]:
list_usernames = list(df_posts.author)
print(list_usernames)

[Redditor(name='Salty_Sidewalks'), Redditor(name='Alexdoh'), Redditor(name='sfvbritguy'), Redditor(name='linguist96'), Redditor(name='Torifyme12'), Redditor(name='ConsciousStop'), Redditor(name='HardworkPanda'), Redditor(name='carlover2393'), Redditor(name='Rofocal07'), Redditor(name='Minezenroll'), Redditor(name='clan_vizsla'), Redditor(name='9lobaldude'), Redditor(name='CordeliaJJ'), Redditor(name='errorists'), Redditor(name='bn911'), Redditor(name='hugglenugget')]


### Exercise 2: Accessing Comments in Reddit

The comments are accessible via the praw method `praw.models.submission.comments`. This returns a `CommentForest`, which lets you know which comments respond to which previous comments by having the struture of a tree.

<div class="alert alert-info">

**Exercise 2.1:** Find the hottest post on the `london` subreddit right now. Do the following afterwards - 
- Access the comments CommentForest
- Use the more_comments() method to access all the top level comments
- Store the comments to a DataFrame
</div>



In [119]:
subreddit = reddit.subreddit('news')

top_post = list(subreddit.hot(limit=1))[0]

In [122]:
top_post.title

'‘The damage is immeasurable’: Woman breaks into Jewish temple, desecrates sanctuary and Torah, then returns to scare children, prosecutors say'

In [125]:
more_to_replace = True
while more_to_replace:
    remaining_more_comments = top_post.comments.replace_more()
    more_to_replace = (len(remaining_more_comments) > 0)

In [131]:
comments = pd.DataFrame([vars(comment) for comment in top_post.comments])

<div class="alert alert-info">

**Exercise 2.2:** Let's look at the content of the comments
- Check which key of the entries contain the content of the comments
- have a look at the replies, score and text of the comments
</div>


In [135]:
comments[['score', 'body', '_replies']].sample(10)

,score,body,_replies
3,154,"I'm going to guess, meth.","(j59vzyz, j59wvmf, j5adwxd, j5a9qum)"
4,100,[removed],"(j5abjcc, j5agmla, j5agzhc, j5ahkxb, j5ah0s6, ..."
2,119,Ok as a Jew here there are a lot of comments p...,"(j5aias5, j5apolp, j5arncy, j5arzvx, j5avbd0)"
10,14,"A Torah scroll can cost several $10,000. Even ...",(j5ac4ix)
13,7,"Synagogue, the word you were looking for was s...","(j5as1nm, j5atjau, j5adyi4)"
1,424,This is going to become more and more common a...,"(j5ad644, j5aokxb, j5aj05p, j5arc5r, j5afviy, ..."
18,1,That bish needs to moisturize. Dusty.,()
8,3,"Your honor, in her defense, my client was just...",()
19,-3,Actually it seems fairly measurable tbh.,()
15,1,You sure that wasn't Kanye doing a Madea impre...,()


### Exercise 3: Accessing User data in Reddit

<div class="alert alert-info">

**Exercise 3.1:** Find comments made by a user `nasa`: limit the query to 1000 entries. 
- Make a dataframe of the entries returned by the API
- convert `created_utc` to a human-readable format
- count the number of comments that `nasa` made on each subreddit. checkout https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html to do so.
</div>

In [182]:
user = reddit.redditor('nasa')
df_user_comments = pd.DataFrame([vars(c) for c in user.comments.new(limit=1000)])
df_user_comments['created_utc'] = pd.to_datetime(df_user_comments['created_utc'], utc=True, unit='s')


In [183]:
df_user_comments[['subreddit', 'created_utc']].sample(10)

,subreddit,created_utc
867,IAmA,2021-11-17 21:03:30+00:00
965,space,2021-09-21 21:07:14+00:00
236,u_nasa,2022-11-09 18:53:09+00:00
413,Futurology,2022-08-19 17:39:31+00:00
780,askscience,2021-12-16 18:13:24+00:00
351,u_nasa,2022-09-20 18:24:25+00:00
161,nasa,2022-12-11 19:37:30+00:00
831,IAmA,2021-11-17 21:51:13+00:00
661,u_nasa,2022-04-08 20:28:05+00:00
281,nasa,2022-10-25 14:38:37+00:00


In [184]:
df_user_comments['subreddit'].value_counts()

askscience          300
u_nasa              241
nasa                216
IAmA                121
space                58
Futurology           35
spaceporn             3
masseffect            2
pics                  2
astrophotography      1
AskReddit             1
jameswebb             1
survivor              1
Name: subreddit, dtype: int64

## Homework: Subreddits of users that posts about OII


<div class="alert alert-info">

- Write a function `subreddit_of_user` that takes in a string input `username` and returns a `list` of unique subreddits on which that `username` comments. 
- Find five posts with score `>10` that contain the keywords - "Oxford Internet Institute" 
- Find out the list of unique authors in the entries returned above
- call `subreddit_of_user` on each of the authors found above
</div>



In [189]:
def subreddits_of_user(username):
    user_comments = reddit.redditor(username)
    try:
        user_comments_subreddits = list(set([c.subreddit for c in user.comments.new(limit=100)]))
    except:
        user_comments_subreddits = []
    return user_comments_subreddits


In [186]:
search_result = reddit.subreddit("all").search("Oxford Internet Institute")
df_posts_oii = pd.DataFrame([vars(p) if (p.score > 10) else {} for p in search_result]).dropna(axis=0,how="all")

list_of_usernames = df_posts_oii.author

In [187]:
list_of_usernames

1     AlgorandFoundation
2                 cysec_
8                 hoyeto
9              agecosmos
13                hargup
14                iRoost
15                HaHard
18                  mvea
19     NutritionResearch
24     fantastic_comment
29          streamingcat
33           PrivacyDude
34     okwherearemypants
48             digitaled
53    labourresearchproj
90           MoonBunnyMB
96           Cianobaxter
97            missbp2189
Name: author, dtype: object

In [190]:
for user in list_of_usernames:
    print(user)
    print(subreddits_of_user(user))

AlgorandFoundation
[]
cysec_
[Subreddit(display_name='AlgorandOfficial'), Subreddit(display_name='algorand'), Subreddit(display_name='Tinyman')]
hoyeto
[Subreddit(display_name='NoShitSherlock'), Subreddit(display_name='MTFSelfieTrain'), Subreddit(display_name='EnoughMuskSpam'), Subreddit(display_name='Buttcoin'), Subreddit(display_name='CryptoReality'), Subreddit(display_name='nottheonion'), Subreddit(display_name='Physics'), Subreddit(display_name='science'), Subreddit(display_name='blendermemes'), Subreddit(display_name='bimbofication'), Subreddit(display_name='SonyAlpha'), Subreddit(display_name='CryptoCurrency'), Subreddit(display_name='blender')]
agecosmos
[Subreddit(display_name='FragrantHarbour'), Subreddit(display_name='ADVChina'), Subreddit(display_name='fucktheccp'), Subreddit(display_name='China_irl'), Subreddit(display_name='CCP_virus'), Subreddit(display_name='interestingasfuck'), Subreddit(display_name='real_China_irl'), Subreddit(display_name='taiwan'), Subreddit(display

<div  class="alert alert-danger">
    <b>Additional Info:</b>
    Last year the course used the Pushshift API instead of the standard Reddit API. I have copied the old material below in case your final project involves Reddit and you want to try Pushshift. In my own testing, it DID NOT WORK, so if you go this direction expect things to get messy.
</div>

### Pushshift API

There are two ways to access this API 
- Plain API through https://api.pushshift.io/.  
- (advanced) Elasticsearch search engine through https://elastic.pushshift.io/. This search engine is designed for fast aggregation and query on big-data. 

The full API is documented here https://github.com/pushshift/api. However, in this exercise, we will learn how to use the plain API through https://api.pushshift.io/ and `psaw`. 

### Querying Pushshift API manually

To use the Pushshift API, we need to know the endpoints that are accessible. Each endpoint serves a specific purpose. There are two endpoints available for this API:
- `/reddit/search/comment` to search for comments
- `/reddit/search/submission` to search for posts

Thus, for example, if we need to search for comments, they can be accessed via https://api.pushshift.io/reddit/search/comment. 

Once we have the correct address, we need a query to search the database. Any data that is sent to an API is included only after a `?` in the URL link. For example, if we need to look up submissions that have word "science" in them, our query will look like:

```
https://api.pushshift.io/reddit/search/submission/?q=science
```

If we click on the above link or copy and paste the above link in any browser, we will see a JSON response from this enddpoint giving us 25 (by default) most recent posts containing the word "science". Each post is in the form of key-value pair. An example response of a post is as follows:

        {
            "all_awardings": [],
            "allow_live_comments": false,
            "author": "Own_Professional_190",
            "author_flair_css_class": null,
            "author_flair_richtext": [],
            "author_flair_text": null,
            "author_flair_type": "text",
            "author_fullname": "t2_jaz08fd5",
            "author_is_blocked": false,
            "author_patreon_flair": false,
            "author_premium": false,
            "awarders": [],
            "can_mod_post": false,
            "contest_mode": false,
            "created_utc": 1644227451,
            "domain": "self.UToledo",
            "full_link": "https://www.reddit.com/r/UToledo/comments/smmgy3/questions_about_transferring/",
            "gildings": {},
            "id": "smmgy3",
            "is_created_from_ads_ui": false,
            "is_crosspostable": true,
            "is_meta": false,
            "is_original_content": false,
            "is_reddit_media_domain": false,
            "is_robot_indexable": true,
            "is_self": true,
            "is_video": false,
            "link_flair_background_color": "",
            "link_flair_richtext": [],
            "link_flair_text_color": "dark",
            "link_flair_type": "text",
            "locked": false,
            "media_only": false,
            "no_follow": true,
            "num_comments": 0,
            "num_crossposts": 0,
            "over_18": false,
            "permalink": "/r/UToledo/comments/smmgy3/questions_about_transferring/",
            "pinned": false,
            "retrieved_on": 1644227461,
            "score": 1,
            "selftext": "Hi, \n\nI am an international student looking for universities to transfer. I heard that Toledo is known for its engineering program and co-op program. I am interested in computer science and data science. Can anybody tell me about general thoughts on university life at Toledo? It can include anything - reputation, class experience, dorm life, life outside of university, and so on. \n\nThank you in advance and stay safe :D",
            "send_replies": true,
            "spoiler": false,
            "stickied": false,
            "subreddit": "UToledo",
            "subreddit_id": "t5_2wpwg",
            "subreddit_subscribers": 182,
            "subreddit_type": "public",
            "thumbnail": "self",
            "title": "Questions about transferring",
            "total_awards_received": 0,
            "treatment_tags": [],
            "upvote_ratio": 1.0,
            "url": "https://www.reddit.com/r/UToledo/comments/smmgy3/questions_about_transferring/"
        }

There are many ways to query this API. Head over to https://github.com/pushshift/api#search-parameters-for-comments to check the paramaters that you can pass to the API to enhance your queries.


### Understanding the JSON response

Here are a few keys returned by the API and what they mean. Most of them are self-explanatory, and which ones are needed will depend heavily on the specific use cases. 

----

| **Key**        	|   	| **Description**                                               	|
|----------------	|---	|---------------------------------------------------------------	|
| _url_          	|   	| url of the `post` or `comment`                                	|
| _author_       	|   	| username of the redditor who created this `post` or `comment` 	|
| _created_utc_  	|   	| time in UTC when this `post` or `comment` was created         	|
| _subreddit_    	|   	| `subreddit` on which this `post` or `comment` was created     	|
| _title_        	|   	| title of the `post`                                           	|
| _selftext_     	|   	| content of the `post` or the `comment`                        	|
| _retrieved_on_ 	|   	| time in UTC when this data was extracted by the Pushshift API 	|

----